# Test Load Daily

Test various aspects of tile-based loading.

## Imports

In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [ ]:
specprod = os.environ['SPECPROD'] = 'daily'
overwrite = True
tiles_patch_date = '20241007'

## Initialize Database

In [ ]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
db.log = get_logger(DEBUG, timestamp=True)
hostname = 'db2-loadbalancer.specprod.production.svc.spin.nersc.org'
# hostname = 'localhost'
postgresql = db.setup_db(schema=specprod, hostname=hostname, username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

## Read tiles file

In [ ]:
original_tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'tiles-daily-patched-with-kibo-{tiles_patch_date}.csv')
current_tiles_file = findfile('tiles', readonly=True)
tiles_table = Table.read(original_tiles_file, format='ascii.csv')
# tiles_table

In [ ]:
current_tiles_file

In [ ]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0))[0]

In [ ]:
%%time
candidate_tiles = db.Tile.convert(tiles_table, row_index=row_index)

## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these. There are also cases where a *tile* has non-zero `EFFTIME_SPEC` but has no *exposures* with non-zero `EFFTIME_SPEC`. At least for now, don't try to load those either.

In [ ]:
original_exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'exposures-daily-patched-with-kibo-{tiles_patch_date}.fits')
current_exposures_file = findfile('exposures', readonly=True)
exposures_table = Table.read(original_exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(original_exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

In [ ]:
%%time
load_tiles = list()
bad_tiles = list()
load_exposures = list()
for new_tile in candidate_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        bad_tiles.append(new_tile)
# load_exposures

In [ ]:
%%time
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_frames

In [ ]:
%%time
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

In [ ]:
%%time
try:
    db.dbSession.add_all(load_exposures)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

In [ ]:
%%time
try:
    db.dbSession.add_all(load_frames)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

## Test tile-based Updates

Useful links:

* https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-queryguide-upsert
* https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#sqlalchemy.dialects.postgresql.Insert.on_conflict_do_update.params.set_
* https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation

In [ ]:
import json
with open(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-cache.json'), 'w') as j:
    json.dump(dict(zip(tiles_table['TILEID'].tolist(), tiles_table['UPDATED'].tolist())), j)

In [ ]:
with open(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-cache.json')) as j:
    tiles_cache = json.load(j)

In [ ]:
update_tiles_table = Table.read(current_tiles_file, format='fits', hdu='TILES')

In [ ]:
cached_tiles = np.array(list(map(int, tiles_cache.keys())))
new_tiles = ~np.in1d(update_tiles_table['TILEID'], cached_tiles)
updated_tiles = np.zeros((len(update_tiles_table), ), dtype=bool)
for tileid in tiles_cache:
    t = int(tileid)
    w = np.where(update_tiles_table['TILEID'] == t)[0]
    assert len(w) == 1
    if update_tiles_table['UPDATED'][w] == tiles_cache[tileid]:
        pass
    elif update_tiles_table['UPDATED'][w] > tiles_cache[tileid]:
        # print("{0} > {1}".format(update_tiles_table['UPDATED'][w], tiles_cache[tileid]))
        updated_tiles[w] = True
    else:
        print("Something weird happened.")
update_tiles_table[new_tiles]

In [ ]:
update_tiles_table[updated_tiles]

In [ ]:
load_new_tiles = db.Tile.convert(update_tiles_table, row_index=new_tiles)
load_updated_tiles = db.Tile.convert(update_tiles_table, row_index=updated_tiles)

### Find exposures associated with new and updated tiles

In [ ]:
update_exposures_table = Table.read(current_exposures_file, format='fits', hdu='EXPOSURES')
update_frames_table = Table.read(current_exposures_file, format='fits', hdu='FRAMES')

In [ ]:
load_tiles = list()
load_exposures = list()
for new_tile in (load_new_tiles + load_updated_tiles):
    row_index = np.where((update_exposures_table['TILEID'] == new_tile.tileid) & (update_exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(update_exposures_table, row_index=row_index)
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((update_exposures_table['TILEID'] == new_tile.tileid))[0]
        print(update_exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        # bad_tiles.append(new_tile)
# load_tiles, load_exposures

# Tiles that *change* to EFFTIME_SPEC = 0 should be removed as in DELETE.
# Keep the database in a state "as if" it had just been reloaded from scratch.

In [ ]:
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(update_frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(update_frames_table, row_index=row_index)

In [ ]:
stmt = db.upsert(load_tiles)
# print(stmt)
db.dbSession.execute(stmt)
db.dbSession.commit()

In [ ]:
# stmt = db.upsert(load_exposures)
# print(stmt)
# db.dbSession.execute(stmt)
# db.dbSession.commit()
db.dbSession.rollback()

In [ ]:
stmt = db.upsert(load_frames)
# print(stmt)
db.dbSession.execute(stmt)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [ ]:
load_tiles[12345]

In [ ]:
tile_index = 0

In [ ]:
load_tiles = db.dbSession.query(db.Tile).filter(db.Tile.tileid == 26055).all()
load_tiles

In [ ]:
load_tiles[tile_index] #, load_tiles[tile_index + 1], load_tiles[-2], load_tiles[-1]

In [ ]:
%%time
potential_targets_table = t.potential_targets(load_tiles[tile_index].tileid)
# potential_targets_table

In [ ]:
%%time
potential_cat = t.potential_photometry(load_tiles[tile_index], potential_targets_table)
# potential_cat

In [ ]:
%%time
potential_targetphot = t.targetphot(potential_cat)

In [ ]:
%%time
potential_tractorphot = t.tractorphot(potential_cat)

In [ ]:
%%time
load_photometry = t.load_photometry(potential_tractorphot)

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [ ]:
%%time
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

### Load the target table

In [ ]:
%%time
load_target = t.load_target(load_tiles[tile_index], potential_targetphot)

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
%%time
load_ztile = t.load_redshift(load_tiles[tile_index])

In [ ]:
l = db.dbSession.query(db.Ztile).filter(db.Ztile.tileid == tile_id).all()

In [ ]:
len(l)

In [ ]:
l[0]

In [ ]:
l[0].z, l[0].zerr, l[0].zwarn, l[0].spectype, l[0].subtype

## Load fiberassign and potential

In [ ]:
%%time
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[tile_index])

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
t.update_primary()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()

In [ ]:
db.dbSession.close()
db.engine.dispose()